### a) Leitura do arquivo de entrada em estruturas de dados apropriadas

In [35]:
from math import inf

from contratos import Contratos


def ler_contratos_de_arquivo(caminho: str) -> Contratos:
  with open('entrada.txt', 'r') as arquivo:
    # Lendo a primeira linha do arquivo texto
    n, m, t = arquivo.readline().split()

    t_taxa = float(t) # Valor da taxa de mudança de fornecedor
    n_meses = int(n) # Quantidade de meses
    m_fornecedores = int(m) # Quantidade de fornecedores

    print(f'(n, m, t) = ({n_meses}, {m_fornecedores}, {t_taxa:.2f})')
    print()

    # Lendo o restante das linhas do arquivo e gerando a matriz base
    # m * n * n
    A = [[[inf for k in range(n_meses)] for j in range(n_meses)] for i in range(m_fornecedores)]

    print(f'Contratos armazenados em matriz: {len(A)}x{len(A[0])}x{len(A[0][0])}')
    print(f'Tamanho total da matriz: {len(A) * len(A[0]) * len(A[0][0])}')

    for linha in arquivo:
        dados = linha.split(' ');
        i_fornecedor, j_inicio, k_fim = map(lambda i: int(i), dados[:-1])
        A[i_fornecedor - 1][j_inicio - 1][k_fim - 1] = float(dados[-1])
    
    return Contratos(A, t_taxa)

c = ler_contratos_de_arquivo('entrada.txt')

(n, m, t) = (120, 100, 100.00)

Contratos armazenados em matriz: 100x120x120
Tamanho total da matriz: 1440000


**b)** Complexidade da estrutura de dados = Θ(m * n^2)

### c) Criar uma função eficiente que retorna o contrato individual, referente ao período completo de n meses, que possui o menor valor

In [36]:
from contratos import Contrato


def get_menor_contrato_periodo(c: Contratos, mes_inicial: int, mes_final: int) -> Contrato:
    menor_contrato = Contrato(c.num_fornecedores, c.num_meses, c.num_meses, inf)
    for forn in range(c.num_fornecedores):
        contrato = c.get_contrato(forn, mes_inicial, mes_final)
        if contrato.valor < menor_contrato.valor:
            menor_contrato = contrato
    return menor_contrato

menor_contrato_periodo_completo = get_menor_contrato_periodo(c, 0, c.num_meses - 1)
print(f'O fornecedor {menor_contrato_periodo_completo.fornecedor + 1} oferece o menor contrato do período completo, que custa: {menor_contrato_periodo_completo.valor:.2f}')


O fornecedor 61 oferece o menor contrato do período completo, que custa: 687.10


**d)** Complexidade de (c): Θ(m)

### e) Criar uma função eficiente que retorna o contrato individual de menor valor do mercado, independente do período a que se refere

In [37]:
def get_menor_contrato(c: Contratos) -> Contrato:
    min_contrato = Contrato(c.num_fornecedores, c.num_meses, c.num_meses, inf)
    for mes in range(c.num_meses):
        contrato = get_menor_contrato_periodo(c, mes, mes)
        if contrato.valor < min_contrato.valor:
            min_contrato = contrato
    return min_contrato

menor_contrato = get_menor_contrato(c)
print(f'O menor contrato é fornecido por {menor_contrato.fornecedor + 1}, do período ({menor_contrato.mes_inicio + 1}, {menor_contrato.mes_fim + 1}) pelo valor de {menor_contrato.valor:.2f}')

O menor contrato é fornecido por 96, do período (4, 4) pelo valor de 10.00


**f)** Complexidade Θ(m * n)

g)

In [38]:
def menor_contrato_periodo_completo(c: Contratos, x_num_meses: int) -> Contrato:
    menor_contrato = Contrato(c.num_fornecedores, c.num_meses, c.num_meses, inf)

    periodo_completo = x_num_meses - 1
    for inicio in range(0, c.num_meses - periodo_completo):
        fim = inicio + periodo_completo
        contrato = get_menor_contrato_periodo(c, inicio, fim)
        if contrato.valor < menor_contrato.valor:
            menor_contrato = contrato
    return menor_contrato

x_periodo = c.num_meses
menor_contrato = menor_contrato_periodo_completo(c, x_periodo)
print(f'Para o período completo de {x_periodo} meses o menor contrato é: {menor_contrato}')


Para o período completo de 120 meses o menor contrato é: Contrato(fornecedor=60, mes_inicio=0, mes_fim=119, valor=687.1)


In [39]:
x_periodo = 4
menor_contrato = menor_contrato_periodo_completo(c, x_periodo)
print(f'Para o período completo de {x_periodo} meses o menor contrato é: {menor_contrato}')


Para o período completo de 4 meses o menor contrato é: Contrato(fornecedor=12, mes_inicio=43, mes_fim=46, valor=20.2)


In [64]:
from contratos import Contrato


def get_menores_contratos_periodo_completo(c: Contratos):
    contratos_return = []
    periodo_completo = c.num_meses - 1
    menor_contrato = Contrato(c.num_fornecedores, c.num_meses, c.num_meses, inf)
    for mes in range(c.num_meses):
        contrato = get_menor_contrato_periodo(c, mes, mes)
        if contrato.valor < menor_contrato.valor:
            contratos_return.append(contrato)
           
    return contratos_return

menor_contrato_periodo_completo = get_menores_contratos_periodo_completo(c)
print(menor_contrato_periodo_completo)


[Contrato(fornecedor=89, mes_inicio=0, mes_fim=0, valor=10.2), Contrato(fornecedor=58, mes_inicio=1, mes_fim=1, valor=10.1), Contrato(fornecedor=46, mes_inicio=2, mes_fim=2, valor=10.1), Contrato(fornecedor=95, mes_inicio=3, mes_fim=3, valor=10.0), Contrato(fornecedor=31, mes_inicio=4, mes_fim=4, valor=10.1), Contrato(fornecedor=80, mes_inicio=5, mes_fim=5, valor=10.0), Contrato(fornecedor=61, mes_inicio=6, mes_fim=6, valor=10.1), Contrato(fornecedor=31, mes_inicio=7, mes_fim=7, valor=10.0), Contrato(fornecedor=72, mes_inicio=8, mes_fim=8, valor=10.0), Contrato(fornecedor=3, mes_inicio=9, mes_fim=9, valor=10.1), Contrato(fornecedor=99, mes_inicio=10, mes_fim=10, valor=10.1), Contrato(fornecedor=56, mes_inicio=11, mes_fim=11, valor=10.1), Contrato(fornecedor=71, mes_inicio=12, mes_fim=12, valor=10.1), Contrato(fornecedor=63, mes_inicio=13, mes_fim=13, valor=10.0), Contrato(fornecedor=17, mes_inicio=14, mes_fim=14, valor=10.0), Contrato(fornecedor=17, mes_inicio=15, mes_fim=15, valor=10.